# Search within 10 verses

Simple notebook POC for searching with in 10 Bible verses.

In [2]:
vers = [
    '我靠主大大地喜乐，因为你们思念我的心如今又发生；你们向来就思念我，只是没得机会。',
    '应当一无挂虑，只要凡事借着祷告、祈求，和感谢，将你们所要的告诉　神。',
    '我在急难中求告耶和华，向我的　神呼求。他从殿中听了我的声音；我在他面前的呼求入了他的耳中。',
    '　神啊，我曾求告你，因为你必应允我；求你向我侧耳，听我的言语。',
    '人的愚昧倾败他的道；他的心也抱怨耶和华。',
    '耶稣对他说：「你若能信，在信的人，凡事都能。」',
    '约书亚对以色列人说：「耶和华－你们列祖的　神所赐给你们的地，你们耽延不去得，要到几时呢？',
    '所罗门说：「耶和华－以色列的　神是应当称颂的！因他亲口向我父大卫所应许的，也亲手成就了。',
    '「因为你富有的时候，不欢心乐意地事奉耶和华－你的　神，',
    '你们当乐意事奉耶和华，当来向他歌唱！'
]
searches = ['挂虑 祈祷', '喜乐 事奉', '求救', '信心 行事']

## Tokenize w/o Stopwords

In [3]:
import jieba
import jieba.posseg as pseg

jieba.enable_paddle()

Paddle enabled successfully......


In [4]:
tokens = list(map(lambda v: pseg.lcut(v, use_paddle=True), vers))
for tk in tokens:
    print(tk)

[pair('我', 'r'), pair('靠', 'p'), pair('主大大地喜乐，', 'nt'), pair('因为', 'p'), pair('你们', 'r'), pair('思念', 'v'), pair('我', 'r'), pair('的', 'u'), pair('心', 'n'), pair('如今', 't'), pair('又', 'd'), pair('发生', 'v'), pair('；', 'v'), pair('你们', 'r'), pair('向来', 'd'), pair('就', 'd'), pair('思念', 'v'), pair('我', 'r'), pair('，', 'v'), pair('只是', 'd'), pair('没得', 'v'), pair('机会', 'n'), pair('。', 'v')]
[pair('应当', 'v'), pair('一无', 'v'), pair('挂虑', 'vn'), pair('，', 'n'), pair('只要', 'c'), pair('凡事', 'n'), pair('借', 'v'), pair('着', 'u'), pair('祷告', 'v'), pair('、', 'n'), pair('祈求', 'v'), pair('，', 'n'), pair('和', 'c'), pair('感谢', 'v'), pair('，', 'n'), pair('将', 'p'), pair('你们', 'r'), pair('所', 'u'), pair('要', 'v'), pair('的', 'u'), pair('告诉', 'v'), pair('\u3000神。', 'nr')]
[pair('我', 'r'), pair('在', 'p'), pair('急难', 'a'), pair('中', 'f'), pair('求告', 'v'), pair('耶和华，', 'PER'), pair('向', 'p'), pair('我', 'r'), pair('的', 'u'), pair('\u3000神', 'n'), pair('呼', 'v'), pair('求', 'v'), pair('。', 'v'), pair('他', 'r'), pai

## Lemmatize

In [5]:
import nltk
from nltk.corpus import wordnet as wn

nltk.download("wordnet")
nltk.download('omw')

[nltk_data] Downloading package wordnet to /Users/Ken/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw to /Users/Ken/nltk_data...
[nltk_data]   Package omw is already up-to-date!


True

In [6]:
syns = wn.synsets('感谢', pos='v', lang='cmn')
for syn in syns:
    print(syn.lemma_names(lang='cmn'))

['感谢', '谢意', '谢谢', '道谢']
['感激', '感谢']


### Similarity

In [7]:
from search import similarity

tests = [
    similarity(('感谢', 'v'), '感动'),
    similarity(('感谢', 'v'), '谢意'),
    similarity(('感谢', 'v'), '谢谢'),
    similarity(('挂虑', 'v'), '挂念'),
    similarity(('挂虑', 'v'), '挂虑'),
    similarity(('祷告', 'v'), '祈求')
]
print(tests)

[0.1, 1.0, 1.0, 0, 1, 0.07142857142857142]


In [8]:
from search import remove_stopwords

print(tokens[0])
print(remove_stopwords(tokens[0]))

[pair('我', 'r'), pair('靠', 'p'), pair('主大大地喜乐，', 'nt'), pair('因为', 'p'), pair('你们', 'r'), pair('思念', 'v'), pair('我', 'r'), pair('的', 'u'), pair('心', 'n'), pair('如今', 't'), pair('又', 'd'), pair('发生', 'v'), pair('；', 'v'), pair('你们', 'r'), pair('向来', 'd'), pair('就', 'd'), pair('思念', 'v'), pair('我', 'r'), pair('，', 'v'), pair('只是', 'd'), pair('没得', 'v'), pair('机会', 'n'), pair('。', 'v')]
[('我', 'r'), ('靠', 'p'), ('主大大地喜乐，', 'nt'), ('因为', 'p'), ('你们', 'r'), ('思念', 'v'), ('我', 'r'), ('的', 'u'), ('心', 'n'), ('如今', 't'), ('又', 'd'), ('发生', 'v'), ('你们', 'r'), ('向来', 'd'), ('就', 'd'), ('思念', 'v'), ('我', 'r'), ('只是', 'd'), ('没得', 'v'), ('机会', 'n')]


In [9]:
from search import sentence_similarity

search_tk = pseg.lcut(searches[0])
print(sentence_similarity(search_tk, tokens[0]))
print(sentence_similarity(search_tk, tokens[1]))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/w0/lx0qzxln5p72qnrg392g7nyc0000gn/T/jieba.cache
Loading model cost 3.225 seconds.
Prefix dict has been built successfully.
0.07142857142857142
1.0


## All Steps Test

In [10]:
for search_term in searches:
    print(f'Search for {search_term}:')
    search_tk = pseg.lcut(search_term)
    for v in vers:
        vers_tk = pseg.lcut(v, use_paddle=True)
        similarity = sentence_similarity(search_tk, vers_tk)
        print(f'Match: {similarity:7.4f} Verse: {v}')
    print()

Search for 挂虑 祈祷:
Match:  0.0714 Verse: 我靠主大大地喜乐，因为你们思念我的心如今又发生；你们向来就思念我，只是没得机会。
Match:  1.0000 Verse: 应当一无挂虑，只要凡事借着祷告、祈求，和感谢，将你们所要的告诉　神。
Match:  0.0714 Verse: 我在急难中求告耶和华，向我的　神呼求。他从殿中听了我的声音；我在他面前的呼求入了他的耳中。
Match:  0.0625 Verse: 　神啊，我曾求告你，因为你必应允我；求你向我侧耳，听我的言语。
Match:  0.0714 Verse: 人的愚昧倾败他的道；他的心也抱怨耶和华。
Match:  0.0625 Verse: 耶稣对他说：「你若能信，在信的人，凡事都能。」
Match:  0.0714 Verse: 约书亚对以色列人说：「耶和华－你们列祖的　神所赐给你们的地，你们耽延不去得，要到几时呢？
Match:  0.0714 Verse: 所罗门说：「耶和华－以色列的　神是应当称颂的！因他亲口向我父大卫所应许的，也亲手成就了。
Match:  0.0357 Verse: 「因为你富有的时候，不欢心乐意地事奉耶和华－你的　神，
Match:  0.0000 Verse: 你们当乐意事奉耶和华，当来向他歌唱！

Search for 喜乐 事奉:
Match:  0.0000 Verse: 我靠主大大地喜乐，因为你们思念我的心如今又发生；你们向来就思念我，只是没得机会。
Match:  0.0000 Verse: 应当一无挂虑，只要凡事借着祷告、祈求，和感谢，将你们所要的告诉　神。
Match:  0.0000 Verse: 我在急难中求告耶和华，向我的　神呼求。他从殿中听了我的声音；我在他面前的呼求入了他的耳中。
Match:  0.0000 Verse: 　神啊，我曾求告你，因为你必应允我；求你向我侧耳，听我的言语。
Match:  0.0000 Verse: 人的愚昧倾败他的道；他的心也抱怨耶和华。
Match:  0.0000 Verse: 耶稣对他说：「你若能信，在信的人，凡事都能。」
Match:  0.0000 Verse: 约书亚对以色列人说：「耶和华－你们列祖的　神所赐给你们的地，你们耽延不去得，要到几时呢？
Match:  0

In [11]:
# Reference: https://bommaritollc.com/2014/06/12/fuzzy-match-sentences-python/
# https://github.com/fxsjy/jieba